In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight


data_seed = 5
train_path = f'../data/custom_train_{data_seed}.csv'
test_path = f'../data/custom_test_{data_seed}.csv'

## LGBM

In [2]:
from LG_Aimers_6th.lgbm_process import lgbm_process

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
train, test = lgbm_process(train, test)
print(train.shape, test.shape)

(205080, 67) (51271, 66)


In [3]:
seed_list = [333] # 333, 777
folds = [10]

In [4]:
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

total_auc, total_acc, total_f1 = [], [], []
test_preds_lgbm = []

for seed in seed_list:
    for k in folds:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        auc_scores, acc_scores,  f1_scores = [], [], []

        for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
            fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_train2 = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
            fold_test = test.copy()

            fold_train, fold_valid = lgbm_process(fold_train, fold_valid, seed=seed)
            fold_train2, fold_test = lgbm_process(fold_train2, fold_test, seed=seed)

            X_train = fold_train.drop(columns=['임신 성공 여부'])
            X_valid = fold_valid.drop(columns=['임신 성공 여부'])
            y_train = fold_train['임신 성공 여부']
            y_valid = fold_valid['임신 성공 여부']

            # Optuna
            lgbm_params = {
                'n_estimators': 1134,
                'learning_rate': 0.009183378614268902,
                'max_depth': 15,
                'num_leaves': 59,
                'min_child_samples': 56,
                'subsample': 0.5894604069264655,
                'colsample_bytree': 0.6305670256882752,
                'reg_alpha': 7.47936987466662,
                'reg_lambda': 0.0010986427203281623,
            }

            model_lgb = LGBMClassifier(
                **lgbm_params,
                verbosity=-1,
                n_jobs=-1,
                random_state=seed,
                early_stopping_rounds=100,
            )

            model_lgb.fit(
                X_train, y_train,
                eval_set=(X_valid, y_valid),
            )

            valid_preds_proba  = model_lgb.predict_proba(X_valid)[:, 1]
            valid_preds_class = model_lgb.predict(X_valid)  # 클래스 예측값 (0 또는 1)

            # AUC, Accuracy, F1-score 계산
            auc_ = roc_auc_score(y_valid, valid_preds_proba)
            acc_ = accuracy_score(y_valid, valid_preds_class)
            f1_ = f1_score(y_valid, valid_preds_class)

            print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {auc_:.7f} | Acc: {acc_:.7f} | F1: {f1_:.7f}")

            auc_scores.append(auc_)
            acc_scores.append(acc_)
            f1_scores.append(f1_)

            total_auc.append(auc_)
            total_acc.append(acc_)
            total_f1.append(f1_)

            test_pred = model_lgb.predict_proba(fold_test)[:, 1]
            test_preds_lgbm.append(test_pred)

        # fold 별 평균 성능 계산
        avg_auc = np.mean(auc_scores)
        avg_acc = np.mean(acc_scores)
        avg_f1 = np.mean(f1_scores)

        print("-" * 80)
        print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")
        print("-" * 80)

val_auc = np.mean(total_auc)
val_acc = np.mean(total_acc)
val_f1 = np.mean(total_f1)

print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f} | Acc: {val_acc:.7f} | F1: {val_f1:.7f}")

Seed[333] Fold 1 | AUC: 0.7416064 | Acc: 0.7469280 | F1: 0.2020295
Seed[333] Fold 2 | AUC: 0.7433026 | Acc: 0.7471718 | F1: 0.1932472
Seed[333] Fold 3 | AUC: 0.7402349 | Acc: 0.7446362 | F1: 0.1803099
Seed[333] Fold 4 | AUC: 0.7401097 | Acc: 0.7458553 | F1: 0.1981538
Seed[333] Fold 5 | AUC: 0.7418990 | Acc: 0.7474644 | F1: 0.1976762
Seed[333] Fold 6 | AUC: 0.7393505 | Acc: 0.7468305 | F1: 0.1997534
Seed[333] Fold 7 | AUC: 0.7434160 | Acc: 0.7448800 | F1: 0.1890887
Seed[333] Fold 8 | AUC: 0.7355309 | Acc: 0.7464892 | F1: 0.1872753
Seed[333] Fold 9 | AUC: 0.7321266 | Acc: 0.7417593 | F1: 0.1827160
Seed[333] Fold 10 | AUC: 0.7434359 | Acc: 0.7444900 | F1: 0.1878487
--------------------------------------------------------------------------------
Seed[333] Average Metrics | AUC: 0.7401013 | Acc: 0.7456505 | F1: 0.1918099
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation

In [5]:
tmp_submission = pd.DataFrame({f'lgbm_{seed_list[0]}': np.mean(test_preds_lgbm, axis=0)})
tmp_submission

,lgbm_333
0,0.066817
1,0.090699
2,0.364770
3,0.242192
4,0.368319
...,...
51266,0.250752
51267,0.001695
51268,0.032542
51269,0.102535


## Catboost

In [6]:
from LG_Aimers_6th.cat_process import cb_all_process
train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = cb_all_process(train, test)

print(train.shape, test.shape)

(205080, 65) (51271, 64)


In [7]:
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

total_auc, total_acc, total_f1 = [], [], []
test_preds_cat = []

is_first = True
for seed in seed_list:
    for k in folds:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        auc_scores, acc_scores,  f1_scores = [], [], []

        for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
            fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_train2 = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
            fold_test = test.copy()

            fold_train, fold_valid = cb_all_process(fold_train, fold_valid)
            fold_train2, fold_test = cb_all_process(fold_train2, fold_test)

            X_train = fold_train.drop(columns=['임신 성공 여부'])
            X_valid = fold_valid.drop(columns=['임신 성공 여부'])
            y_train = fold_train['임신 성공 여부']
            y_valid = fold_valid['임신 성공 여부']

            cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

            if is_first:
                cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
                print(f'범주형 변수: {len(cat_features)}개 \n {cat_features} \n')

                num_features = [col for col in X_train.columns if col not in cat_features + ['임신 성공 여부']]
                print(f'수치형 변수: {len(num_features)}개 \n {num_features} \n')
                is_first = False

            weights_sqrt = np.sqrt(compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train))

            # Catboost 모델 초기화
            model_cat = CatBoostClassifier(iterations=2000, learning_rate=0.05, random_seed=seed,
                                       loss_function='Logloss', eval_metric='Logloss', class_weights=weights_sqrt,
                                       cat_features=cat_features, thread_count=-1)

            model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)

            valid_preds_proba = model_cat.predict_proba(X_valid)[:, 1]
            valid_preds_class = model_cat.predict(X_valid)  # 클래스 예측값 (0 또는 1)

            # AUC, Accuracy, F1-score 계산
            auc_ = roc_auc_score(y_valid, valid_preds_proba)
            acc_ = accuracy_score(y_valid, valid_preds_class)
            f1_ = f1_score(y_valid, valid_preds_class)

            print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {auc_:.7f} | Acc: {acc_:.7f} | F1: {f1_:.7f}")

            auc_scores.append(auc_)
            acc_scores.append(acc_)
            f1_scores.append(f1_)

            total_auc.append(auc_)
            total_acc.append(acc_)
            total_f1.append(f1_)

            test_pred = model_cat.predict_proba(fold_test)[:, 1]
            test_preds_cat.append(test_pred)

        # fold 별 평균 성능 계산
        avg_auc = np.mean(auc_scores)
        avg_acc = np.mean(acc_scores)
        avg_f1 = np.mean(f1_scores)

        print("-" * 80)
        print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")
        print("-" * 80)

val_auc = np.mean(total_auc)
val_acc = np.mean(total_acc)
val_f1 = np.mean(total_f1)

print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f} | Acc: {val_acc:.7f} | F1: {val_f1:.7f}")

범주형 변수: 18개 
 ['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '배아 생성 주요 이유', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합'] 

수치형 변수: 46개 
 ['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신'] 

0:	learn: 0.6747170	test: 0.6747491	best: 0.6747491 (

In [8]:
tmp_submission[f'cat_{seed_list[0]}'] = np.mean(test_preds_cat, axis=0)
tmp_submission

,lgbm_333,cat_333
0,0.066817,0.112119
1,0.090699,0.147462
2,0.364770,0.499679
3,0.242192,0.405690
4,0.368319,0.496530
...,...,...
51266,0.250752,0.339533
51267,0.001695,0.005670
51268,0.032542,0.067882
51269,0.102535,0.173954


In [9]:
tmp_submission['row_avg'] = tmp_submission.mean(axis=1)
tmp_submission

,lgbm_333,cat_333,row_avg
0,0.066817,0.112119,0.089468
1,0.090699,0.147462,0.119081
2,0.364770,0.499679,0.432225
3,0.242192,0.405690,0.323941
4,0.368319,0.496530,0.432425
...,...,...,...
51266,0.250752,0.339533,0.295143
51267,0.001695,0.005670,0.003683
51268,0.032542,0.067882,0.050212
51269,0.102535,0.173954,0.138244


In [10]:
from LG_Aimers_6th.cal_auc import calculate_auc

score = calculate_auc(tmp_submission['row_avg'], seed=data_seed)
print(f'[seed {data_seed}]: {score}')

[seed 6]: 0.740052283283912
